In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, Accuracy, MeanAbsoluteError
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense, Concatenate, Input, Dropout,  LSTM, Flatten, Embedding
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from keras.models import Sequential
import keras_nlp
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


Using TensorFlow backend


In [2]:
df = pd.read_csv('../data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
7954,2,['W'],False,['Instant'],['none'],Prevent all damage that would be dealt to crea...,['no keywords'],0,0,common,False,5175,True,0.08
5214,5,['G'],False,['Creature'],"['Phyrexian', 'Wurm']",Trample As an additional cost to cast green pe...,['Trample'],6,6,rare,False,10509,True,2.34
13657,2,['G'],False,['Creature'],"['Human', 'Wizard']",2: Target red or white creature gains trample ...,['no keywords'],2,1,uncommon,False,4972,True,0.08
3335,3,"['G', 'W']",False,['Creature'],"['Centaur', 'Cleric']","When Centaur Healer enters the battlefield, yo...",['no keywords'],3,3,common,False,8955,True,0.03
18336,4,['W'],False,['Creature'],"['Human', 'Scout']",Outlast 1White,['Outlast'],2,5,common,False,7604,True,0.02


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cmc          25545 non-null  int64  
 1   colors       25545 non-null  object 
 2   legendary    25545 non-null  bool   
 3   types        25545 non-null  object 
 4   sub_types    25545 non-null  object 
 5   oracle_text  25523 non-null  object 
 6   keywords     25545 non-null  object 
 7   power        25545 non-null  int64  
 8   toughness    25545 non-null  int64  
 9   rarity       25545 non-null  object 
 10  reserved     25545 non-null  bool   
 11  released_at  25545 non-null  int64  
 12  booster      25545 non-null  bool   
 13  usd          25489 non-null  float64
dtypes: bool(3), float64(1), int64(4), object(6)
memory usage: 2.2+ MB


In [4]:
df = df[['oracle_text', 'usd']]
df.head(3)

,oracle_text,usd
0,Equipped creature gets +2/+2. Equip 3,0.02
1,Flying When Aarakocra Sneak enters the battlef...,0.06
2,Trample Mark of Chaos Ascendant — During your ...,2.81


In [5]:
df = df.dropna(subset=['oracle_text', 'usd'])
df = df.reset_index(drop=True)

In [6]:
df['oracle_text'].isna().value_counts(), df['usd'].isna().value_counts()

(oracle_text
 False    25476
 Name: count, dtype: int64,
 usd
 False    25476
 Name: count, dtype: int64)

In [7]:
X, y = df['oracle_text'], df['usd']

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
total_words = len(tokenizer.word_index) + 1

In [9]:
X = tokenizer.texts_to_sequences(X)

max_sequence_length = max([len(seq) for seq in X])
X = pad_sequences(X, maxlen=max_sequence_length, padding='pre')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
def build_model():

    metrics = [MeanAbsoluteError()]

    model = Sequential()
    model.add(Embedding(input_dim=total_words, output_dim=150, input_length=max_sequence_length))
    model.add(LSTM(150))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))

    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=metrics)

    return model


In [12]:
model = build_model()

In [13]:
def get_history():
    es = EarlyStopping(patience=10, restore_best_weights=True)


    history = model.fit(
        X_train,
        y_train,
        epochs=100,
        validation_data=(X_test, y_test),
        callbacks=[es],
        verbose=1,
        workers=4,
        use_multiprocessing=True
    )

    return history

In [14]:
get_history()

Epoch 1/100
637/637 [==============================] - 96s 148ms/step - loss: 1530.3091 - mean_absolute_error: 3.8639 - val_loss: 3508.2324 - val_mean_absolute_error: 4.0949
Epoch 2/100
637/637 [==============================] - 81s 128ms/step - loss: 1525.9735 - mean_absolute_error: 3.6459 - val_loss: 3503.4165 - val_mean_absolute_error: 4.6605
Epoch 3/100
637/637 [==============================] - 89s 139ms/step - loss: 1517.0326 - mean_absolute_error: 3.8480 - val_loss: 3509.5989 - val_mean_absolute_error: 4.4095
Epoch 4/100
637/637 [==============================] - 1145s 2s/step - loss: 1505.7631 - mean_absolute_error: 3.9263 - val_loss: 3516.6104 - val_mean_absolute_error: 4.2183
Epoch 5/100
637/637 [==============================] - 1363s 2s/step - loss: 1485.6469 - mean_absolute_error: 3.8054 - val_loss: 3533.9924 - val_mean_absolute_error: 4.6250
Epoch 6/100
637/637 [==============================] - 96s 150ms/step - loss: 1435.1980 - mean_absolute_error: 3.8715 - val_loss: 35

In [16]:
model.evaluate(X_test, np.array(y_test))

160/160 [==============================] - 5s 29ms/step - loss: 3503.4165 - mean_absolute_error: 4.6533


[3503.41650390625, 4.65326452255249]